In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
from numpy import array

print(tf.__version__)

2.11.0


Importo cartella di tutti gli esercizi matematici, zippata in dati.zip

In [2]:
from zipfile import ZipFile
file_name = "dati.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done1')

Done1


Importo varie librerie, tra cui di rilievo numpy e pandas e ovviamente tensorflow->keras

In [3]:
from tensorflow import keras
import pandas as pd
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
%matplotlib inline
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorboard.plugins import projector
plt.style.use('ggplot')


In [4]:
tutti_gli_esercizi=[] #contenitore di tutte gli esercizi, sia di test che di training, mi servirà per fare embedding(non voglio che a stessi indici del vocabolario corrispondano parole diverse)


Leggo tutti i file di testo della cartella esercizi matematici, tramite un for itero ogni stringa di ogni file di testo e la metto in contenitori (1 per i dati di training, 1 per i dati di test e 1 per il validation set.

Le etichette saranno poi un indice da 1 a 7 con la seguente mappatura:

1) 3D geometric figures in spatial diagrams 

2) arithmetic

3) crypto-arithmetic

4) geometric figures in spatial diagrams OR puzzle

5) spatial reasoning

6) numbers in spatial diagrams

7) temporal reasoning

In [5]:

lista_tutti_esercizi=[]
lista_labels=[]
esercizio=""
'''
2 vettori, uno per i testi e uno per le corrispettive etichette.  
'''
iter=1
while(iter<=70):
  esercizio=""
  with open("/content/Esercizi matematici_/train/3D geometric figures in spatial diagrams/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels.append(1)
  
  with open("/content/Esercizi matematici_/train/arithmetic/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels.append(2)

  with open("/content/Esercizi matematici_/train/crypto-arithmetic/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi.append(esercizio)
  esercizio=""
  lista_labels.append(3)

  if(iter<45 or iter>49):
    with open("/content/Esercizi matematici_/train/geometric figures in spatial diagrams OR puzzle/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+=line
    lista_tutti_esercizi.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels.append(4)

  if(iter<16 or iter>17):
    with open("/content/Esercizi matematici_/train/spatial reasoning/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+=line
    lista_tutti_esercizi.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels.append(5)

  with open("/content/Esercizi matematici_/train/numbers in spatial diagrams/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels.append(6)

  with open("/content/Esercizi matematici_/train/temporal reasoning/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels.append(7)

  iter+=1


In [6]:
'''dati di test'''
lista_tutti_esercizi_test=[]
lista_labels_test=[]
esercizio=""
iter=81
while(iter<=116):
  esercizio=""
  if(iter<=97):
    with open("/content/Esercizi matematici_/test/3D geometric figures in spatial diagrams/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+= line
    lista_tutti_esercizi_test.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels_test.append(1)

  if(iter<=111):
    with open("/content/Esercizi matematici_/test/arithmetic/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+= line
    lista_tutti_esercizi_test.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels_test.append(2)

  if(iter<=112):
    with open("/content/Esercizi matematici_/test/crypto-arithmetic/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+= line
    lista_tutti_esercizi_test.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels_test.append(3)

  if(iter<100):
    with open("/content/Esercizi matematici_/test/geometric figures in spatial diagrams OR puzzle/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+= line
    lista_tutti_esercizi_test.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels_test.append(4)

  if(iter<116):
    with open("/content/Esercizi matematici_/test/spatial reasoning/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+= line
    lista_tutti_esercizi_test.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels_test.append(5)

  if(iter<=100):
    with open("/content/Esercizi matematici_/test/numbers in spatial diagrams/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+= line
    lista_tutti_esercizi_test.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels_test.append(6)

  if(iter<=101):
    with open("/content/Esercizi matematici_/test/temporal reasoning/"+str(iter)+".txt", "r") as f:
      parole_esercizio= f.readlines()
      for line in parole_esercizio:
        esercizio+= line
    lista_tutti_esercizi_test.append(esercizio)
    tutti_gli_esercizi.append(esercizio)
    esercizio=""
    lista_labels_test.append(7)

  iter+=1

In [7]:
lista_tutti_esercizi_validation=[]
lista_labels_validation=[]
esercizio=""
'''

VALIDATION

'''
iter=71
while(iter<=80):
  esercizio=""
  with open("/content/Esercizi matematici_/validation/3D geometric figures in spatial diagrams/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi_validation.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels_validation.append(1)
  
  with open("/content/Esercizi matematici_/validation/arithmetic/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi_validation.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels_validation.append(2)

  with open("/content/Esercizi matematici_/validation/crypto-arithmetic/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi_validation.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels_validation.append(3)


  with open("/content/Esercizi matematici_/validation/geometric figures in spatial diagrams OR puzzle/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi_validation.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels_validation.append(4)


  with open("/content/Esercizi matematici_/validation/spatial reasoning/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi_validation.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels_validation.append(5)

  with open("/content/Esercizi matematici_/validation/numbers in spatial diagrams/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi_validation.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels_validation.append(6)

  with open("/content/Esercizi matematici_/validation/temporal reasoning/"+str(iter)+".txt", "r") as f:
    parole_esercizio= f.readlines()
    for line in parole_esercizio:
      esercizio+=line
  lista_tutti_esercizi_validation.append(esercizio)
  tutti_gli_esercizi.append(esercizio)
  esercizio=""
  lista_labels_validation.append(7)

  iter+=1


Inserisco i dati in un dataFrame, DataFrame di Pandas, così che mi si faciliti la loro gestione, ottenendo un'indicizzazione che ricorda un foglo excel

In [8]:
train_d={'esercizi':lista_tutti_esercizi,'etichette':lista_labels}
test_d={'esercizi':lista_tutti_esercizi_test,'etichette':lista_labels_test}

train_data = pd.DataFrame(data=train_d)
test_data = pd.DataFrame(data=test_d)

print(train_data)
print(test_data)

                                              esercizi  etichette
0    La panchina che vedete in figura è stata\ncost...          1
1    Jacopo ha ottenuto 35 punti lanciando 4 frecce...          2
2    Le cinque caselle bianche vanno completate con...          3
3    Quanti sono i percorsi diversi che si possono ...          4
4    Il tappeto rettangolare grigio che vedete in\n...          5
..                                                 ...        ...
478  Scrivete nelle caselle lungo il perimetro del\...          3
479  Nove bambini — Alessandro, Bernardo, Clotilde,...          4
480  Vista dall’alto, una torta è un disco perfetto...          5
481  Scriviamo due numeri diversi di una cifra (da ...          6
482  La lumaca Lilly vive in un bel giardino ma ama...          7

[483 rows x 2 columns]
                                              esercizi  etichette
0    Emi dispone di 120 piccoli cubi tutti uguali: ...          1
1    Moltiplica per 4 il numero del mese di nascita.

In [9]:
train_data['esercizi'] = train_data['esercizi'].apply(lambda x: x.lower()) # to lowercase
train_data['esercizi'] = train_data['esercizi'].apply((lambda x: re.sub('[^a-z\s]', '', x))) # remove all characters that are not in a-z
train_data['esercizi'] = train_data['esercizi'].replace(r'\n',' ', regex=True)


test_data['esercizi'] = test_data['esercizi'].apply(lambda x: x.lower()) # to lowercase
test_data['esercizi'] = test_data['esercizi'].apply((lambda x: re.sub('[^a-z\s]', '', x))) # remove all characters that are not in a-z
test_data['esercizi'] = test_data['esercizi'].replace(r'\n',' ', regex=True)

for i in range(len(tutti_gli_esercizi)):
  tutti_gli_esercizi[i] = tutti_gli_esercizi[i].lower()
  tutti_gli_esercizi[i] = re.sub('[^a-z\s]', '', tutti_gli_esercizi[i])
  tutti_gli_esercizi[i] = tutti_gli_esercizi[i].replace('\n',' ')

for i in range(len(lista_tutti_esercizi_validation)):
  lista_tutti_esercizi_validation[i] = lista_tutti_esercizi_validation[i].lower()
  lista_tutti_esercizi_validation[i] = re.sub('[^a-z\s]', '', lista_tutti_esercizi_validation[i])
  lista_tutti_esercizi_validation[i] = lista_tutti_esercizi_validation[i].replace('\n',' ')

print(train_data)
print(test_data)
print(tutti_gli_esercizi)
print(lista_tutti_esercizi_validation)

                                              esercizi  etichette
0    la panchina che vedete in figura  stata costru...          1
1    jacopo ha ottenuto  punti lanciando  freccette...          2
2    le cinque caselle bianche vanno completate con...          3
3    quanti sono i percorsi diversi che si possono ...          4
4    il tappeto rettangolare grigio che vedete in f...          5
..                                                 ...        ...
478  scrivete nelle caselle lungo il perimetro del ...          3
479  nove bambini  alessandro bernardo clotilde dav...          4
480  vista dallalto una torta  un disco perfetto co...          5
481  scriviamo due numeri diversi di una cifra da  ...          6
482  la lumaca lilly vive in un bel giardino ma ama...          7

[483 rows x 2 columns]
                                              esercizi  etichette
0    emi dispone di  piccoli cubi tutti uguali  blu...          1
1    moltiplica per  il numero del mese di nascita .

Scelta della dimensione dell'embedding: 100, che è convenzionale per Word2Vec

In [10]:
embed_dim = 100 # size of Word2Vec embeddings


Conteggio di tutte le parole che occorrono tra tutto il dataset(training+validation+test).

lista_tutte_parole= lista di tutte le parole, anche ripetute.

parole_training_no_dup = set, qui non sono dublicate le parole. Mi darà la grandezza del dizionario.

In [16]:
lista_tutte_parole= []
for sentence in tutti_gli_esercizi:
    words = sentence.split()
    for word in words:
        lista_tutte_parole.append(word)

parole_training_no_dup= set(lista_tutte_parole)
print(parole_training_no_dup)

# number of unique words
words_number = len(parole_training_no_dup)

vocab_size = words_number # qui stabilisco grandezza dizionario


print(words_number)


{'fatto', 'dadi', 'rettangolariquanti', 'effettuarsi', 'estremit', 'uninfinit', 'luscita', 'attraversano', 'tracciare', 'entrambe', 'federazione', 'abcdef', 'acque', 'riportando', 'be', 'perfezionista', 'fora', 'identica', 'soddisfatto', 'manca', 'grigia', 'aiutarlo', 'popolazione', 'dubbio', 'sapori', 'abxyz', 'magico', 'pescato', 'stato', 'lintera', 'cactus', 'lavorer', 'esista', 'paese', 'riuscito', 'diverso', 'i', 'conti', 'colora', 'scriverle', 'vernice', 'inversi', 'imballaggio', 'scarpe', 'ottobre', 'geocity', 'cambiarne', 'laboratorio', 'allunita', 'avevo', 'aver', 'indica', 'animale', 'affiancate', 'strategia', 'lascia', 'scriverne', 'quadratidisegnate', 'disegnato', 'esaurire', 'pross', 'mattina', 'spiccioli', 'volino', 'novit', 't', 'petalo', 'gare', 'rimane', 'domando', 'nuovo', 'meno', 'ulteriormente', 'accumulando', 'che', 'ab', 'dialogo', 'numeratore', 'spigoli', 'finiscono', 'lunione', 'raffigurato', 'descrive', 'cigm', 'visitato', 'tipo', 'ai', 'possiede', 'larchitetto

Ora vado a tokenizzare il dizionario, fornendo padding sulla massima dimensione degli esercizi

In [17]:

tokenizer = Tokenizer(num_words=words_number, split=' ')
#tokenizer.fit_on_texts(train_data['esercizi'].values) # <= non posso fare, avrei per stesso indice del vocab parole diverse
tokenizer.fit_on_texts(tutti_gli_esercizi) # <= ATTENTION HERE!

X_train = tokenizer.texts_to_sequences(train_data['esercizi'].values)
X_train = pad_sequences(X_train)  # Padding to make sequences of same length
Y_train = pd.get_dummies(train_data['etichette']).values

print('train')
print(X_train)
print(Y_train)

# same for test data
#tokenizer.fit_on_texts(test_data['sentence'].values)
X_test = tokenizer.texts_to_sequences(test_data['esercizi'].values)
#X_test = pad_sequences(X_test)
X_test = pad_sequences(X_test,185)
Y_test = pd.get_dummies(test_data['etichette']).values

X_val= tokenizer.texts_to_sequences(lista_tutti_esercizi_validation)
X_val= pad_sequences(X_val,185)
Y_val = pd.get_dummies(lista_labels_validation)

print('test')
print(X_test)
print(Y_test)

# dictionary -> (word: integer)
wordindex_dict = tokenizer.word_index





train
[[   0    0    0 ...    3   48 1014]
 [   0    0    0 ...   26 2413  617]
 [   0    0    0 ...   18  233  125]
 ...
 [   0    0    0 ... 1591   19  552]
 [   0    0    0 ...   27  974  150]
 [   0    0    0 ... 3768   19 1198]]
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
test
[[   0    0    0 ...   78  382  277]
 [   0    0    0 ...  968    5  552]
 [   0    0    0 ... 1592   75  764]
 ...
 [   0    0    0 ...  968    5  552]
 [   0    0    0 ... 1964  496   64]
 [   0    0    0 ... 4350   51 4351]]
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


In [19]:

model_gamma1 = keras.Sequential()
model_gamma1.add(keras.layers.Embedding(vocab_size, embed_dim, input_length=X_train.shape[1]))
model_gamma1.add(keras.layers.GlobalAveragePooling1D())
model_gamma1.add(keras.layers.Dense(16, activation=tf.nn.relu))
model_gamma1.add(Dense(7, activation='softmax'))

model_gamma1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 185, 100)          460500    
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                1616      
                                                                 
 dense_1 (Dense)             (None, 7)                 119       
                                                                 
Total params: 462,235
Trainable params: 462,235
Non-trainable params: 0
_________________________________________________________________


In [20]:
model_gamma1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
history = model_gamma1.fit(X_train,
                    Y_train,
                    epochs=69,
                    batch_size=50,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Epoch 1/69
10/10 [==============================] - 2s 136ms/step - loss: 1.9476 - accuracy: 0.1449 - val_loss: 1.9448 - val_accuracy: 0.1571
Epoch 2/69
10/10 [==============================] - 0s 25ms/step - loss: 1.9420 - accuracy: 0.1884 - val_loss: 1.9424 - val_accuracy: 0.1571
Epoch 3/69
10/10 [==============================] - 0s 21ms/step - loss: 1.9381 - accuracy: 0.1925 - val_loss: 1.9401 - val_accuracy: 0.1429
Epoch 4/69
10/10 [==============================] - 0s 20ms/step - loss: 1.9330 - accuracy: 0.2319 - val_loss: 1.9372 - val_accuracy: 0.1857
Epoch 5/69
10/10 [==============================] - 0s 17ms/step - loss: 1.9273 - accuracy: 0.3313 - val_loss: 1.9337 - val_accuracy: 0.2857
Epoch 6/69
10/10 [==============================] - 1s 107ms/step - loss: 1.9200 - accuracy: 0.3851 - val_loss: 1.9291 - val_accuracy: 0.2714
Epoch 7/69
10/10 [==============================] - 0s 34ms/step - loss: 1.9116 - accuracy: 0.4099 - val_loss: 1.9240 - val_accuracy: 0.2714
Epoch 8/69


In [22]:
model_gamma1.predict(X_test)

6/6 [==============================] - 0s 5ms/step


array([[9.20779407e-01, 2.17005610e-02, 1.13334725e-04, ...,
        1.88128580e-03, 1.98128808e-04, 8.89063464e-04],
       [5.11407200e-03, 2.45276496e-01, 9.81492642e-03, ...,
        1.98496953e-01, 3.56543772e-02, 5.00801921e-01],
       [3.33967693e-02, 5.90514839e-02, 3.17869544e-01, ...,
        2.41195291e-01, 1.91699088e-01, 3.09743937e-02],
       ...,
       [2.52515059e-02, 2.94704386e-03, 1.82171613e-02, ...,
        8.00900161e-01, 2.82474328e-03, 2.47472315e-03],
       [1.33619428e-01, 2.66130418e-01, 4.98267263e-03, ...,
        2.83196062e-01, 4.81403898e-03, 5.15807196e-02],
       [7.37866713e-03, 3.36340144e-02, 6.24174550e-02, ...,
        7.84237623e-01, 3.65441591e-02, 2.10481547e-02]], dtype=float32)

In [23]:
results_gamma1 = model_gamma1.evaluate(X_test, Y_test)
print('test loss, test acc:', results_gamma1)

6/6 [==============================] - 2s 340ms/step - loss: 1.0380 - accuracy: 0.6686
test loss, test acc: [1.0379579067230225, 0.668571412563324]


In [24]:
from gensim.models import Word2Vec
import numpy as np

# Define your dataset
dataset = parole_training_no_dup

# Train a Word2Vec model on the dataset
model = Word2Vec(dataset, min_count=1, size=100, window=5)

# Create a matrix of embeddings
embeddings_matrix = np.zeros((len(dataset), model.vector_size))
for i, sentence in enumerate(dataset):
    sentence_embedding = np.zeros(model.vector_size)
    n_words = 0
    for word in sentence:
        if word in model:
            sentence_embedding += model[word]
            n_words += 1
    if n_words > 0:
        sentence_embedding /= n_words
    embeddings_matrix[i] = sentence_embedding

# Print the embeddings matrix
print(embeddings_matrix)

<ipython-input-24-519009d9882f>:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in model:
<ipython-input-24-519009d9882f>:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  sentence_embedding += model[word]


[[-0.24627786  0.29530758  0.09390699 ...  0.14294146  0.02524463
   0.02142262]
 [-0.24191707  0.29204823  0.09272763 ...  0.14047533  0.02731945
   0.01771208]
 [-0.24869491  0.29817311  0.09423622 ...  0.14370907  0.02758391
   0.01908958]
 ...
 [-0.24447583  0.29223825  0.09488011 ...  0.14133081  0.02843981
   0.0182209 ]
 [-0.24801799  0.29958556  0.09183061 ...  0.1415661   0.02476754
   0.01967303]
 [-0.24871808  0.29811319  0.09203356 ...  0.14352457  0.02563009
   0.02078937]]


lavoro ora con matrice di embeddings pre addestrata, sempre con schema del modello gamma 1

In [61]:
modello_delta = Sequential()
modello_delta.add(Embedding(vocab_size, embed_dim, weights=[embeddings_matrix], # Here we use the pre-trained embeddings
                    input_length=X_train.shape[1], trainable=True))
modello_delta.add(keras.layers.GlobalAveragePooling1D())
modello_delta.add(keras.layers.Dense(16, activation=tf.nn.relu))
modello_delta.add(Dense(7, activation='softmax'))

modello_delta.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 185, 100)          460500    
                                                                 
 global_average_pooling1d_4   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_13 (Dense)            (None, 16)                1616      
                                                                 
 dense_14 (Dense)            (None, 7)                 119       
                                                                 
Total params: 462,235
Trainable params: 462,235
Non-trainable params: 0
_________________________________________________________________


In [62]:
modello_delta.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [63]:
history = modello_delta.fit(X_train,
                    Y_train,
                    epochs=73,
                    batch_size=50,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Epoch 1/73
10/10 [==============================] - 5s 243ms/step - loss: 1.9485 - accuracy: 0.1449 - val_loss: 1.9463 - val_accuracy: 0.1429
Epoch 2/73
10/10 [==============================] - 1s 74ms/step - loss: 1.9429 - accuracy: 0.1449 - val_loss: 1.9435 - val_accuracy: 0.1429
Epoch 3/73
10/10 [==============================] - 0s 26ms/step - loss: 1.9381 - accuracy: 0.2940 - val_loss: 1.9417 - val_accuracy: 0.3714
Epoch 4/73
10/10 [==============================] - 0s 36ms/step - loss: 1.9342 - accuracy: 0.3147 - val_loss: 1.9401 - val_accuracy: 0.2714
Epoch 5/73
10/10 [==============================] - 2s 213ms/step - loss: 1.9300 - accuracy: 0.3602 - val_loss: 1.9378 - val_accuracy: 0.2857
Epoch 6/73
10/10 [==============================] - 2s 218ms/step - loss: 1.9255 - accuracy: 0.2961 - val_loss: 1.9354 - val_accuracy: 0.1714
Epoch 7/73
10/10 [==============================] - 0s 18ms/step - loss: 1.9210 - accuracy: 0.1822 - val_loss: 1.9323 - val_accuracy: 0.1429
Epoch 8/73

In [64]:
modello_delta.predict(X_test)

6/6 [==============================] - 0s 4ms/step


array([[8.0889302e-01, 5.4116198e-03, 3.1544664e-03, ..., 1.6081874e-01,
        6.9456026e-03, 2.8278059e-04],
       [1.8189548e-02, 5.5158502e-01, 4.7081131e-02, ..., 5.2792672e-02,
        8.0751337e-02, 2.3550148e-01],
       [9.7621180e-02, 1.0799896e-01, 3.5628870e-01, ..., 1.2896731e-01,
        1.7879818e-01, 3.8070407e-02],
       ...,
       [3.6425775e-01, 3.0708604e-04, 2.4190322e-03, ..., 5.0371140e-01,
        5.7420903e-03, 1.5835484e-04],
       [1.7571589e-01, 6.7170352e-02, 3.1792998e-02, ..., 3.9982054e-01,
        2.6125221e-02, 8.1211664e-02],
       [1.8912481e-01, 1.6745046e-02, 7.0125304e-02, ..., 4.1198242e-01,
        6.7921884e-02, 1.1880589e-02]], dtype=float32)

In [65]:
results_delta = modello_delta.evaluate(X_test, Y_test)
print('test loss, test acc:', results_delta)

6/6 [==============================] - 0s 4ms/step - loss: 0.9841 - accuracy: 0.7143
test loss, test acc: [0.984061598777771, 0.7142857313156128]


Stampo ora il valore dei pesi ottenuti dopo l'addestramento dal modello delta

In [38]:
weights=[]
weights = tf.Variable(modello_delta.layers[0].get_weights()[0][0:])
print(weights)

<tf.Variable 'Variable:0' shape=(4605, 100) dtype=float32, numpy=
array([[-0.3668298 ,  0.26437476,  0.13644516, ...,  0.16300407,
         0.03271981,  0.04143437],
       [-0.2711465 ,  0.3276586 ,  0.19852254, ...,  0.34107912,
         0.24363956, -0.14333184],
       [-0.5808889 ,  0.07986503,  0.20606129, ...,  0.14678146,
         0.3637267 , -0.16667904],
       ...,
       [-0.24447584,  0.29223824,  0.09488011, ...,  0.14133081,
         0.02843981,  0.0182209 ],
       [-0.24801798,  0.29958555,  0.0918306 , ...,  0.1415661 ,
         0.02476754,  0.01967303],
       [-0.24871808,  0.2981132 ,  0.09203356, ...,  0.14352457,
         0.02563009,  0.02078937]], dtype=float32)>


Stampo la differenza tra pesi ottenuti dal training del modello e la matrice di embedding iniziale

In [44]:
diff_emb= weights - embeddings_matrix
print(diff_emb)

tf.Tensor(
[[-0.12055194 -0.03093281  0.04253818 ...  0.02006261  0.00747519
   0.02001175]
 [-0.02922943  0.03561038  0.10579491 ...  0.20060378  0.2163201
  -0.16104391]
 [-0.33219403 -0.21830806  0.11182507 ...  0.0030724   0.3361428
  -0.18576862]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]], shape=(4605, 100), dtype=float32)


capiamo ora con quale accuratezza il modello delta farebbe predizione sui dati di training e sui dati di test e validation

In [45]:
modello_delta_2 = Sequential()
modello_delta_2.add(Embedding(vocab_size, embed_dim, weights=[embeddings_matrix], # Here we use the pre-trained embeddings
                    input_length=X_train.shape[1], trainable=False))
modello_delta_2.add(keras.layers.GlobalAveragePooling1D())
modello_delta_2.add(keras.layers.Dense(16, activation=tf.nn.relu))
modello_delta_2.add(Dense(7, activation='softmax'))

modello_delta_2.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 185, 100)          460500    
                                                                 
 global_average_pooling1d_2   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 16)                1616      
                                                                 
 dense_5 (Dense)             (None, 7)                 119       
                                                                 
Total params: 462,235
Trainable params: 1,735
Non-trainable params: 460,500
_________________________________________________________________


In [46]:
modello_delta_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
history = modello_delta_2.fit(X_train,
                    Y_train,
                    epochs=73,
                    batch_size=50,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Epoch 1/73
10/10 [==============================] - 2s 100ms/step - loss: 1.9527 - accuracy: 0.1077 - val_loss: 1.9479 - val_accuracy: 0.1429
Epoch 2/73
10/10 [==============================] - 0s 8ms/step - loss: 1.9479 - accuracy: 0.1387 - val_loss: 1.9464 - val_accuracy: 0.1429
Epoch 3/73
10/10 [==============================] - 0s 8ms/step - loss: 1.9469 - accuracy: 0.1449 - val_loss: 1.9463 - val_accuracy: 0.1429
Epoch 4/73
10/10 [==============================] - 0s 9ms/step - loss: 1.9465 - accuracy: 0.1408 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 5/73
10/10 [==============================] - 0s 7ms/step - loss: 1.9461 - accuracy: 0.1449 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 6/73
10/10 [==============================] - 0s 7ms/step - loss: 1.9461 - accuracy: 0.1449 - val_loss: 1.9462 - val_accuracy: 0.1429
Epoch 7/73
10/10 [==============================] - 0s 7ms/step - loss: 1.9458 - accuracy: 0.1449 - val_loss: 1.9462 - val_accuracy: 0.1429
Epoch 8/73
10/10 [

In [49]:
modello_delta_2.predict(X_test)

6/6 [==============================] - 1s 156ms/step


array([[0.1442812 , 0.14522946, 0.14621362, ..., 0.14137372, 0.1452607 ,
        0.14381982],
       [0.1442975 , 0.14522403, 0.14619039, ..., 0.14137082, 0.14524701,
        0.14383005],
       [0.14423804, 0.1452429 , 0.14624614, ..., 0.14139836, 0.14527513,
        0.14380379],
       ...,
       [0.14435875, 0.14520505, 0.1461402 , ..., 0.14133728, 0.14522235,
        0.14385757],
       [0.14426343, 0.14523146, 0.14621395, ..., 0.1413916 , 0.14525695,
        0.14381902],
       [0.14426196, 0.14523876, 0.14622234, ..., 0.1413874 , 0.1452619 ,
        0.1438163 ]], dtype=float32)

In [52]:
results_delta_2 = modello_delta_2.evaluate(X_test, Y_test)
print('test loss, test acc:', results_delta_2)

6/6 [==============================] - 0s 36ms/step - loss: 1.9443 - accuracy: 0.1829
test loss, test acc: [1.9442572593688965, 0.18285714089870453]


Modello delta_3:
  Provo a costruire nuovi layer, con struttura romboidale, per capire se facendo rimanere gli embedding inziali di word2vec, addestrando solo i pesi dei layer successivi a quelli dello strato di embedding, posso ottenere un modello che mi predice con un accuratezza accettabile.

In [53]:
modello_delta_3 = Sequential()
modello_delta_3.add(Embedding(vocab_size, embed_dim, weights=[embeddings_matrix], # Here we use the pre-trained embeddings
                    input_length=X_train.shape[1], trainable=False))
modello_delta_3.add(keras.layers.GlobalAveragePooling1D())
modello_delta_3.add(keras.layers.Dense(7, activation=tf.nn.relu))
modello_delta_3.add(keras.layers.Dense(14, activation=tf.nn.relu))
modello_delta_3.add(keras.layers.Dense(28, activation=tf.nn.relu))
modello_delta_3.add(keras.layers.Dense(14, activation=tf.nn.relu))
modello_delta_3.add(Dense(7, activation='softmax'))

modello_delta_3.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 185, 100)          460500    
                                                                 
 global_average_pooling1d_3   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 7)                 707       
                                                                 
 dense_7 (Dense)             (None, 14)                112       
                                                                 
 dense_8 (Dense)             (None, 28)                420       
                                                                 
 dense_9 (Dense)             (None, 14)                406       
                                                      

In [54]:
modello_delta_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [55]:
history = modello_delta_3.fit(X_train,
                    Y_train,
                    epochs=70,
                    batch_size=50,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Epoch 1/70
10/10 [==============================] - 2s 89ms/step - loss: 1.9470 - accuracy: 0.1449 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 2/70
10/10 [==============================] - 0s 9ms/step - loss: 1.9462 - accuracy: 0.1449 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 3/70
10/10 [==============================] - 0s 9ms/step - loss: 1.9462 - accuracy: 0.1346 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 4/70
10/10 [==============================] - 0s 7ms/step - loss: 1.9461 - accuracy: 0.1449 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 5/70
10/10 [==============================] - 0s 10ms/step - loss: 1.9459 - accuracy: 0.1449 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 6/70
10/10 [==============================] - 0s 7ms/step - loss: 1.9459 - accuracy: 0.1449 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 7/70
10/10 [==============================] - 0s 9ms/step - loss: 1.9459 - accuracy: 0.1449 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 8/70
10/10 [

modello delta_4
provo a eliminare lo strato di GlobalAveragPooling. lo sostituisco con 4 strati LSTM (2 bidirezionale e 2 unidirezionali)


In [56]:
lstm_out=10
modello_delta_4 = Sequential()
modello_delta_4.add(Embedding(vocab_size, embed_dim, weights=[embeddings_matrix], # Here we use the pre-trained embeddings
                    input_length=X_train.shape[1], trainable=False))
modello_delta_4.add(Bidirectional(LSTM(lstm_out, return_sequences=True, # this option must be used if we have more LSTM
                             activation='tanh', dropout=0.2, recurrent_dropout=0.2)))
modello_delta_4.add(LSTM(lstm_out, return_sequences=True))
modello_delta_4.add(Bidirectional(LSTM(lstm_out, return_sequences=True, # this option must be used if we have more LSTM
                             activation='tanh', dropout=0.2, recurrent_dropout=0.2)))
modello_delta_4.add(LSTM(lstm_out))
modello_delta_4.add(keras.layers.Dense(16, activation=tf.nn.relu))
modello_delta_4.add(Dense(7, activation='softmax'))

modello_delta_4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 185, 100)          460500    
                                                                 
 bidirectional (Bidirectiona  (None, 185, 20)          8880      
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 185, 10)           1240      
                                                                 
 bidirectional_1 (Bidirectio  (None, 185, 20)          1680      
 nal)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 10)                1240      
                                                                 
 dense_11 (Dense)            (None, 16)               

In [57]:
modello_delta_4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
history = modello_delta_4.fit(X_train,
                    Y_train,
                    epochs=70,
                    batch_size=50,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Epoch 1/70
10/10 [==============================] - 33s 1s/step - loss: 1.9462 - accuracy: 0.1532 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 2/70
10/10 [==============================] - 12s 1s/step - loss: 1.9462 - accuracy: 0.1346 - val_loss: 1.9463 - val_accuracy: 0.1429
Epoch 3/70
10/10 [==============================] - 11s 1s/step - loss: 1.9461 - accuracy: 0.1573 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 4/70
10/10 [==============================] - 12s 1s/step - loss: 1.9460 - accuracy: 0.1449 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 5/70
10/10 [==============================] - 11s 1s/step - loss: 1.9460 - accuracy: 0.1284 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 6/70
10/10 [==============================] - 12s 1s/step - loss: 1.9460 - accuracy: 0.1325 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 7/70
10/10 [==============================] - 12s 1s/step - loss: 1.9459 - accuracy: 0.1429 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 8/70
10/10 [==

**In sintesi, mantenendo i valori degli embedding invariati, e provando vari modelli, anche con stessa struttura del gamma 1, non si riesce a costruire un modello che sappia predire gli esercizi matematici.
I valori restituiti dalla matrice di embedding dell'Word2Vec non riescono a coprire la rilevanza dei significati delle parole immerse in un contesto più tecnico, in termini di differenza di specificità dell'argomento proposto dall'esercizio. Sono infatti valori più generali e inerenti a un linguaggio non troppo dettagliato per questo specifico problema.**


**Bisogna quindi apprendere dei nuovi valori che ci qualificano le parole in ambiente più specifico, addestrando quindi necessariamente nuovi pesi per gli embeddings.**

Risultati:

con il modello gamma 1 il massimo risultato ottenuto in termini di accuratezza è stato con 70 epoche, avendo ottenuto un'accuratezza di:

*   0.9772 sui dati di training
*   0.7714 sul validation set (ultima epoca)
*   0.72 sui dati di test


con il modello delta 1 il massimo valore di accuratezza (considerando preponderanti i dati di test) è stato ottenuto per 72 epoche.
accuracy:

*   0.9607 sui dati di training
*   0.6143 sul validation set (ultima epoca)
*   0.7085 sui dati di test

Il fatto di addestrare il modello partendo da una matrice di embeddings, ottenuta tramite word2vec, non porta a maggiori livelli di accuratezza, anzi.
